In [9]:
from utils import *
from modules import *
from missing_sync import build_schema
%matplotlib widget
datapicker = create_datapicker(path=r'Z:\Exp_4-outdoor_walk\lisbon\sourcedata\data\OE001',schema=build_schema, calibrate_ubx_to_harp=False)
display(datapicker)

ModuleNotFoundError: No module named 'modules'

In [ ]:
import importlib
importlib.reload(for_climate)

for path in participants_info:
    try:        
        # Load the dataset
        data_path  = path['session_folder']
        datapicker = create_datapicker(path=data_path,schema=build_schema, calibrate_ubx_to_harp=False)
        dataset    = load_dataset(datapicker.selected_path, ubx=True, unity=False, calibrate_ubx_to_harp=False, schema=build_schema)

        # Create geodata
        geodata = dataset.to_geoframe()

        # Process geodata
        geodata['time'] = geodata.index.to_pydatetime()
        geodata         = tidy_geodata(geodata)
        geodata         = add_environmental_metrics(geodata)
    except Exception as e:
        print(f"An unexpected error occurred for participant code '{path['participant_code']}': {e}")
        print("Most likely needs to have an updated EEG .nedf file")
        continue

### 1. Load dataset without GPS (para Andre: está a dar erro no load)

datasets without GPS (13 in total):
OE101001
OE101002
OE102002
OE102003
OE104003
OE104004
OE101012
OE106005
OE106011
OE106010
OE106016
OE106021
OE106032

In [ ]:
dataset = load_dataset(datapicker.selected_path, ubx=True, unity=False, calibrate_ubx_to_harp=False, schema=build_schema)

In [ ]:
datapicker.geodata


In [ ]:
exploremap(datapicker.geodata, column='tk_airquality_iaqindex_value')

In [ ]:
datapicker = create_datapicker(path=r'Z:\Exp_4-outdoor_walk\lisbon\sourcedata\data\OE003', schema=build_schema)
display(datapicker)

# END

In [ ]:
from modules import *
from pluma.schema.outdoor import build_schema
%matplotlib widget

### 1. Load dataset

In [ ]:
datapicker = create_datapicker(path=r'Z:\Exp_4-outdoor_walk\lisbon\sourcedata\data\OE003', schema=build_schema)
display(datapicker)

### 2. Explore Dataset Maps

In [ ]:
exploremap(datapicker.geodata)

### 3. Explore Dataset Streams

In [ ]:
print(datapicker.dataset.streams.EEG.data)

datapicker.dataset.streams.EEG.data["np_time"]



In [ ]:
datapicker.dataset.streams.EEG.data



In [ ]:
plot_traces({
    'accelX': datapicker.dataset.streams.Accelerometer.data["Accl_X"],
    'temp (C)': datapicker.dataset.streams.TK.AirQuality.Temperature.data/100,
    'angleX': datapicker.dataset.streams.Accelerometer.data["Orientation_X"],
    'altitude': datapicker.dataset.georeference.elevation,
    'iaq': datapicker.dataset.streams.TK.AirQuality.IAQIndex.data,
    'eeg': datapicker.dataset.streams.EEG.data.np_eeg,
})

## Export to IGOT

In [ ]:
import os
import pandas as pd
from openpyxl import Workbook
from pythermalcomfort.models import utci
import numpy as np

# ---------------------------------------------------------------------
# Initialize the Excel workbook and sheet


# Path information
#logdata    = r'Z:\Exp_4-outdoor_walk\lisbon\sourcedata\supp\log'

# To store the mean values for each session
# all_means = []

# ---------------------------------------------------------------------
# FUNCTIONS

radiant_temp_status = 0  # Default to 0 if not found or if all zeros

# Generate the sessions.tsv file
geodata = datapicker.geodata  # Retrieve geodata






# Define custom parameters
humidity = geodata['tk_humidity_humidity_value'] / 100  # in fraction
wind_speed = np.sqrt(geodata['atmos_northwind_value']**2 + geodata['atmos_eastwind_value']**2)  # m/s (~2.5 m of elevation)
temp_atmos = geodata['atmos_airtemperature_value']  # in ºC
temp_tk = geodata['tk_airquality_temperature_value'] / 100  # in ºC
temp_tk_ptc = geodata['tk_ptc_airtemp_value'] / 100  # in ºC
temp_radiant = geodata['tk_thermocouple_temperature_value'] / 100  # in ºC

# Assign custom parameters to the geodata attribute
geodata['humidity'] = humidity
geodata['wind_speed'] = wind_speed
geodata['temp_atmos'] = temp_atmos
geodata['temp_tk'] = temp_tk
geodata['temp_tk_ptc'] = temp_tk_ptc
geodata['temp_radiant'] = temp_radiant

# Compute the UTCI
geodata['utci'] = utci(tdb=temp_atmos, tr=temp_radiant, v=wind_speed, rh=humidity)








# Get GPS coordinates and integrate them into geodata
coords = datapicker.geodata.geometry.get_coordinates(include_z=True)
coords.rename(columns={'y': 'latitude', 'x': 'longitude', 'z': 'elevation'}, inplace=True)
geodata = geodata.join(coords).drop(columns=['geometry'])

# Check if the radiant temperature column exists and contains non-zero values
if 'tk_thermocouple_temperature_value' in geodata:
    if geodata['tk_thermocouple_temperature_value'].sum() != 0:
        radiant_temp_status = 1  # Non-zero values exist

# Save the full geodata to an Excel file in the log directory
#log_folder = os.path.join(logdata, f"sub-{participant_name}", f"ses-{session_name}")
#os.makedirs(log_folder, exist_ok=True)
# geodata_file = os.path.join(log_folder, f"sub-{participant_name}_ses-{session_name}_geodata.xlsx")
# geodata.to_excel(geodata_file, index=False)

# Compute the mean of all numerical columns
# numeric_means = geodata.mean(numeric_only=True).to_dict()
# numeric_means['Participant'] = participant_name
# numeric_means['Session'] = session_name
# all_means.append(numeric_means)  # Append to the global all_means list

# Log the result


# def create_geodata(geodata):
#    """Compute UTCI across the whole time series."""
    


# ---------------------------------------------------------------------
# MAIN SCRIPT
# processed_participants = 0

# for participant_folder in os.listdir(sourcedata):
#     if participant_folder.startswith("OE"):
#         participant_path = os.path.join(sourcedata, participant_folder)
#         for session_folder in os.listdir(participant_path):
#             session_path = os.path.join(participant_path, session_folder)
#             if os.path.isdir(session_path):
#process_session( participant_folder, session_folder)

        # processed_participants += 1
        # if processed_participants >= 2:
        #     break  # Stop after processing 2 participants

# Save the Excel file with the updated status in the log directory
# result_file = os.path.join(logdata, "session_processing_results.xlsx")
# wb.save(result_file)

# Save the mean values to a final CSV file
# if all_means:
#     final_means_df = pd.DataFrame(all_means)
#     final_means_file = os.path.join(logdata, "final_means.csv")
#     final_means_df.to_csv(final_means_file, index=False)
#     print(f"Final CSV file with mean values saved to: {final_means_file}")
# else:
#     print("No valid data found to compute mean values.")

In [ ]:
datapicker.geodata

In [ ]:
strs = datapicker.selected_path.split('\\')
strs[len(strs)-1]

In [ ]:
logdata = r'c:\EXPORT'
final_means_file = os.path.join(logdata, strs[len(strs)-1]+'.csv')
datapicker.geodata.to_csv(final_means_file, index=True)

### 4. Export Dataset to OGC API

In [ ]:
record = DatasetRecord(datapicker.dataset, datapicker.geodata, properties=RecordProperties(
    title='<City> Outdoor Walk: <CityRegion> Subject <ID>',
    description='Outdoor walk data collection',
    license='CC BY-NC 4.0',
    tool='Bonsai',
    keywords=['<City>', 'Outdoor', 'Walk', 'Microclimate', 'Biosignals'],
    contacts=[
        Contact(
            name='Your Name',
            institution='Your Institution',
            email='youremail@yourdomain.com'
        )
    ],
    themes=[]
))

In [ ]:
rpath = Path(record.id)
export_geoframe_to_geojson(datapicker.geodata, rpath.with_suffix('.geojson'))
with open(rpath.with_suffix('.json'), 'w') as f:
    f.write(record.to_json())